# [Managing Growing Projects with Packages, Crates, and Modules](https://doc.rust-lang.org/book/ch07-00-managing-growing-projects-with-packages-crates-and-modules.html)
I like my name better.

I'm keen to learn how Rust managaes packages and scope. 
 - How do you choose what methods or variables should be exposed?
 - What naming conventions should I be following?
 - How do I separate internal and external imports?
 - How do these 'crates' work?
 
## [Packages and Crates](https://doc.rust-lang.org/book/ch07-01-packages-and-crates.html#packages-and-crates)
Crates are:
 - The smallest unit, a single file.
 - Either binary or library.
    - Binaries crates have a main function and work independently.
    - Library crates do not have a main function and work with a program.
 - Often 'crate' will be used interchangably with 'library' in Rust.
 - Binary creates have a 'crate root', the entry file.

Packages are:
 - One or more crates bundled together.
 - Work to provide a set of functionality.
 - Have a 'Cargo.toml' file to setup the build.
 - Can contain many binary crates **but only one library crate**.
 - Must always have at least one crate.
 - The default root crate is the 'src/main.rs' file.
    - Additional files can be put in 'src/bin/'.
 - The default library crate is 'src/lib.rs'
 - The default package name is the name of the parent directory.
 
Seems pretty straightforward so far.

I'll try creating my own package when I have all the details.

## [Defining Modules to Control Scope and Privacy](https://doc.rust-lang.org/book/ch07-02-defining-modules-to-control-scope-and-privacy.html)

Rules for modules:
 * Crates start from the crate root of 'src/lib.rs' or 'src/main.rs'.
 * Modules are declared as `mod module_name;` and are defined as:
    * The code in the {} brackets following the declaration (if present).
    * The 'src/module_name.rs' file.
    * The 'src/module_name/mod.rs' file.
 * It doesn't seem like inline modules would usually be very desirable? You'd want to split up the code.
 * Module files can contain own submodules.
 * Submodules are accessed with: `crate::module_name::sub_module_name`.
 * Everything in a module is private by default (including submodules) unless the `pub` keyword is used.
    * It's really nice to have that by default.
    * What about members of an enum or struct, is `pub` all or nothing?
 * The `use` keyword lets you shorten long paths.
    * I already did this in the previous program adding `use std::f64::consts::PI;` lets you use just 'PI' by itself.

## [Grouping Related Code in Modules](https://doc.rust-lang.org/book/ch07-02-defining-modules-to-control-scope-and-privacy.html#grouping-related-code-in-modules)

They gave the example contents of a library's 'src/lib.rs' file:
```rust
mod front_of_house {
    mod hosting {
        fn add_to_waitlist() {}

        fn seat_at_table() {}
    }

    mod serving {
        fn take_order() {}

        fn serve_order() {}

        fn take_payment() {}
    }
}
```

Is the filename of the library taken into account? 

Does it preceed the module names in the namespace?

It sounds like it.

They have confirmed that `crate` is the implicit parent module. 

I started to make a library modular for the maths of AM modulation.

However, I did not yet know how to link the files together, so I put the module inline for now.

Some thoughts:
 * Very strong encapsulation by default.
   * As mention, all methods are automatically private.
   * Making the parent class public does not make its members public.
   * Neither does it make the members of the parent struct public.
   * Is public/private the only distinction? Or are there friend classes of sort?
 * You can also do `use module::x as y` if you don't like the default name.
   * I did `use std::f64::consts::PI as Π;` because it makes mathematical formulae look a lot nicer.
      * I'm not sure how to feel about the lack of automatic promotion of ints to floats.
 * Unicode systems worked properly in names.

I wrote nearly three hundred lines for this modulation library:
 * This was probably more of an undertaking that in would be in a more familiar language.
 * The validation system was admittedly overly complex. I was too keen to practice Rust's safety features.
 * It's nice to be able to use the proper unicode symbols in formulae on my own code.
    * In any kind of colaboration there are complains from the less... mathematically inclined.
    * It's so much easier to read something like `π` or `ω` than having a screen full of `very_long_variable_names`.
 * `match` works much better in practice that I expected. It's a better version of switch!
 * I'm getting a lot of errors about unused function calls.
    * Not much that can be done. Most of this needs to be present, but isn't immediately used.
 * WE NEED TESTING.
    * It's impractical to build any decent sized module without a strong test system
       * Still yet to learn that framework.
 * It would be best to abandon this effort for now. It's a little out of hand from a practice program.
    * I could manually comb through it and look for bugs like in ye olde days, but there still a lot to learn.
    
   There was first discussion in the chapter which boiled down to your standard public/private separate talk.


## [Starting Relative Paths with super](https://doc.rust-lang.org/book/ch07-03-paths-for-referring-to-an-item-in-the-module-tree.html#starting-relative-paths-with-super)

I was wondering when we'd get to this.

In most languages you'd have learnt super class addressing much earlier.

It was one of the first thing I learnt in Java tutorials.

Though doesn't seem like Rust really uses inheritance? So I suppose it's not that important?

Here's the example they gave:
```rust
fn deliver_order() {}

mod back_of_house {
    fn fix_incorrect_order() {
        cook_order();
        super::deliver_order();
    }

    fn cook_order() {}
}
```

So it's just used in the namespace?

Well it seems like a situation where you can't use absolute paths.

But perhaps this `super` keyword is not that super in this case. 



## [Making Structs and Enums Public](https://doc.rust-lang.org/book/ch07-03-paths-for-referring-to-an-item-in-the-module-tree.html#making-structs-and-enums-public)

So I can make my stuct members public on a case by case basis?

```rust
mod back_of_house {
    pub struct Breakfast {
        pub toast: String,
        seasonal_fruit: String,
    }

    impl Breakfast {
        pub fn summer(toast: &str) -> Breakfast {
            Breakfast {
                toast: String::from(toast),
                seasonal_fruit: String::from("peaches"),
            }
        }
    }
}
```

This looks like it will provide very granular control.

Which seems nice, but won't that get annoying to use so many "pub"s?

I'm not sure.

Ah, it says all enum members are public by default.

`use` imports aren't inherited.
```rust
mod front_of_house {
    pub mod hosting {
        pub fn add_to_waitlist() {}
    }
}
```

This works:
```rust
use crate::front_of_house::hosting;

pub fn eat_at_restaurant() {
    hosting::add_to_waitlist();
}
```

This doesn't:
```rust
mod customer {
    pub fn eat_at_restaurant() {
        hosting::add_to_waitlist();
    }
}
```

This is an... interesting rule.

I'll see how it goes.



## [Providing New Names with the as Keyword](https://doc.rust-lang.org/book/ch07-04-bringing-paths-into-scope-with-the-use-keyword.html#providing-new-names-with-the-as-keyword)

You can import as a different name like with `import x as y` in Python.

```rust
use std::fmt::Result;
use std::io::Result as IoResult;

fn function1() -> Result {
    // --snip--
}

fn function2() -> IoResult<()> {
    // --snip--
}
```

You don't have to do that `from x import y as z` nonsense.

Seems like a good interpretation then.




<style> p { max-width: 500px; } </style>
## [Using External Packages](https://doc.rust-lang.org/book/ch07-04-bringing-paths-into-scope-with-the-use-keyword.html#using-external-packages)

So we can add something like: 
```toml
rand = "0.9.0-beta.1"
``` 
To the "Cargo.toml" file to have `cargo` automatically download and build dependency packages (crates).

But that's not required to use packages from the standard library's `std` package.



<style> p { max-width: 500px; } </style>
You can use `package::{item_a, item_b}` syntax to add multiple items from the same path.

This:
```rust
use std::cmp::Ordering;
use std::io;
```

Can be converted to this:
```rust
use std::{cmp::Ordering, io};
```

And this:
```rust
use std::io;
use std::io::Write;
```

Can be converted to this:
```rust
use std::io::{self, Write};
```
(`std::io` and ```std::io::Write``` are now in scope)

Pretty neat. I like how `self` is used here to import both the parent and a subcomponent.

There's also a glob syntax:
```rust
use std::collections::*;
```

Yeah... I don't think I'll use this one very often.

It's really bad practice.

The suggested use case is test code, but even then I'd prefer to fully name the import items needed.



<style> p { max-width: 500px; } </style>
Apparently the information on how to run tests is still five chapters away...😑


<style> p { max-width: 500px; } </style>

## [Separating Modules into Different Files](https://doc.rust-lang.org/book/ch07-05-separating-modules-into-different-files.html#separating-modules-into-different-files)

When splitting code into different files you can use the `crate::(filename)::(module)` syntax to import them.

This code is put in 'src/front_of_house.rs':
```rust
pub mod hosting {
    pub fn add_to_waitlist() {}
}
```

And is imported into the main module (src/lib.rs) like so:
```rust
mod front_of_house;

pub use crate::front_of_house::hosting;

pub fn eat_at_restaurant() {
    hosting::add_to_waitlist();
}
```

`mod` is used to declare files that are part of the crate.

`crate` is used to include those declared files.

You only need to use `mod` once for each file!

Interestingly this can be used to make submodules.

To move the `hosting` module to a subdirectory you can put this in 'src/front_of_house.rs':
```rust
pub mod hosting;
```

Then hosting will be loaded from 'src/front_of_house/hosting.rs'.

(or alternatively 'src/front_of_house/hosting/mod.rs')

Pretty neat!

Though this just repeats what we learnt a few chapters ago.